This notebook is aiming at generating a csv file for further label validation. As we suspect there are discrepancies between original label vs. SME label, this notebook will assist us by generating a csv that could easily help us check if there are indeed discrepancies.

Prerequisite:

1) csv/xlsx format original label file

2) csv/xlsx format SME label file

3) (optional) csv/xlsx format aws predicted result as reference

Output:

A csv file named label_revisit will be saved locally, which can be used to validate if there are discrepancies between SME and original training labels.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
# Specify original label .xlsx filename and location
key_file_addr = "Batch3Label.xlsx"

In [3]:
# Load label file
key_file = pd.read_excel(key_file_addr)

In [4]:
# Check how many labels are in label file 
label_from_all = key_file['CRFieldName']
label_from_all.unique()

array(['admission_order', 'discharge_summary', 'history_physical',
       'admit_type', 'admission_reason', 'mar', 'labs', 'flowsheets',
       'radiology', 'consult', 'physician_query', 'procedures',
       'progress_notes', 'emergency_department', 'case_management',
       'discharge_disposition', 'nutritional_assessment'], dtype=object)

In [5]:
# Drop empty labels records
valid_label_df = key_file.dropna(subset = ['User entered Pages'])
valid_label_df

CR_ID                                   CRFileName        CRFieldName  \
0        1382    OCR_1382_1003700000000633142_11068501.pdf    admission_order   
2        1382    OCR_1382_1003700000000633142_11068501.pdf   history_physical   
3        5217    OCR_5217_1003700000000642572_11072823.pdf         admit_type   
4        7132    OCR_7132_1003700000000680026_11074907.pdf  discharge_summary   
5        1382    OCR_1382_1003700000000633142_11068501.pdf   admission_reason   
...       ...                                          ...                ...   
35139  112864  OCR_112864_1003700000000633020_11248108.pdf            consult   
35140  112864  OCR_112864_1003700000000633020_11248108.pdf                mar   
35141  113136  OCR_113136_1003700000000697811_11249401.pdf   history_physical   
35142  113136  OCR_113136_1003700000000697811_11249401.pdf               labs   
35144  114686  OCR_114686_1003700000000675009_11251999.pdf   history_physical   

      User entered Pages Cannon Identified Pages  
0                     29                       1  
2                  43-47                   43-47  
3                    2,3                     2,3  
4                262-278            17-59,61-113  
5                 1,4,44               1,4,20,44  
...                  ...                     ...  
35139                  0                   29-30  
35140              12-24           12-24,222-224  
35141              40-46                     NaN  
35142            203-212           58-60,203-212  
35144              79-85               79-86,173  

[29434 rows x 5 columns]

In [6]:
# Keep only the columns of interests, which are the filename, label pages and class name
original_label_df = valid_label_df.copy()
original_label_df = original_label_df[["CRFileName", "User entered Pages", "CRFieldName"]]
original_label_df

CRFileName User entered Pages  \
0        OCR_1382_1003700000000633142_11068501.pdf                 29   
2        OCR_1382_1003700000000633142_11068501.pdf              43-47   
3        OCR_5217_1003700000000642572_11072823.pdf                2,3   
4        OCR_7132_1003700000000680026_11074907.pdf            262-278   
5        OCR_1382_1003700000000633142_11068501.pdf             1,4,44   
...                                            ...                ...   
35139  OCR_112864_1003700000000633020_11248108.pdf                  0   
35140  OCR_112864_1003700000000633020_11248108.pdf              12-24   
35141  OCR_113136_1003700000000697811_11249401.pdf              40-46   
35142  OCR_113136_1003700000000697811_11249401.pdf            203-212   
35144  OCR_114686_1003700000000675009_11251999.pdf              79-85   

             CRFieldName  
0        admission_order  
2       history_physical  
3             admit_type  
4      discharge_summary  
5       admission_reason  
...                  ...  
35139            consult  
35140                mar  
35141   history_physical  
35142               labs  
35144   history_physical  

[29434 rows x 3 columns]

In [7]:
# Change filename to same format 
for i in range(len(original_label_df)):
    orig_filename = original_label_df['CRFileName'].iloc[i]
    filename = original_label_df['CRFileName'].iloc[i].split('_')[-2] + '_' + original_label_df['CRFileName'].iloc[i].split('_')[-1]
    original_label_df['CRFileName'].iloc[i] = filename

In [8]:
# Change column name to better illustrate
original_label_df.rename(columns={'User entered Pages' : 'Original Label'}, inplace=True)
original_label_df

CRFileName Original Label        CRFieldName
0      1003700000000633142_11068501.pdf             29    admission_order
2      1003700000000633142_11068501.pdf          43-47   history_physical
3      1003700000000642572_11072823.pdf            2,3         admit_type
4      1003700000000680026_11074907.pdf        262-278  discharge_summary
5      1003700000000633142_11068501.pdf         1,4,44   admission_reason
...                                 ...            ...                ...
35139  1003700000000633020_11248108.pdf              0            consult
35140  1003700000000633020_11248108.pdf          12-24                mar
35141  1003700000000697811_11249401.pdf          40-46   history_physical
35142  1003700000000697811_11249401.pdf        203-212               labs
35144  1003700000000675009_11251999.pdf          79-85   history_physical

[29434 rows x 3 columns]

In [9]:
# Load the excel file of SME label 
lexicon_file_addr = "Lexicon_Training_Data_Batch5.xlsx"
lexicon_file = pd.read_excel(lexicon_file_addr)

# Select only the batch that are of interest (batch 5 in this example)
lexicon_file = lexicon_file[lexicon_file['BatchName'] == 'AWS HUM DRG BATCH 5 - Set1']

In [10]:
# Check how many labels are in SME label file 
label_from_all = lexicon_file['CRFieldName']
label_from_all.unique()

array(['procedures', 'ed', 'history_physical', 'physician_query',
       'radiology', 'admit_type', 'admission_order', 'admission_reason',
       'discharge_summary', 'discharge_disposition', 'case_management',
       'nutrition_assessment', 'flowsheets', 'mar', 'consult', 'labs',
       'progress_notes'], dtype=object)

In [11]:
# Drop empty labels records
valid_label_df = lexicon_file.dropna(subset = ['User entered Pages'])

In [12]:
# Rename the SME label dataframe and check its class number
sme_label_df = valid_label_df.copy()
len(sme_label_df['CRFieldName'].unique())

17

In [13]:
# Keep the columns of interests and rename the SME label column to better clarify
sme_label_df = sme_label_df[["CRFileName", "User entered Pages", "CRFieldName"]]
sme_label_df.rename(columns={'User entered Pages' : 'SME Label'}, inplace=True)

In [14]:
# Check common file
common_files = set(original_label_df.CRFileName).intersection(sme_label_df.CRFileName)
len(common_files)

33

In [15]:
# filter out uncommon file
original_label_df_common = original_label_df[original_label_df.CRFileName.isin(common_files)]

In [16]:
# Only keep the class that we want to validate from original label, in this example we want to validate NA/DD/CM
original_label_df_combine = pd.DataFrame()
original_label_df_common_na = original_label_df_common[original_label_df_common['CRFieldName'] == "nutritional_assessment"]
original_label_df_common_dd = original_label_df_common[original_label_df_common['CRFieldName'] == "discharge_disposition"]
original_label_df_common_ca = original_label_df_common[original_label_df_common['CRFieldName'] == "case_management"]
original_label_df_combine = pd.concat([original_label_df_combine, original_label_df_common_na, original_label_df_common_dd, original_label_df_common_ca], axis=0, ignore_index=True)

In [17]:
# Only keep the class that we want to validate from SME label, in this example we want to validate NA/DD/CM
sme_label_df_combine = pd.DataFrame()
# Make sure the class name are the same as SME's class name
sme_label_df_common_na = sme_label_df[sme_label_df['CRFieldName'] == "nutrition_assessment"]
sme_label_df_common_dd = sme_label_df[sme_label_df['CRFieldName'] == "discharge_disposition"]
sme_label_df_common_ca = sme_label_df[sme_label_df['CRFieldName'] == "case_management"]
sme_label_df_combine = pd.concat([sme_label_df_combine, sme_label_df_common_na, sme_label_df_common_dd, sme_label_df_common_ca], axis=0, ignore_index=True)
# Replace SME class name by original name to unify label name
sme_label_df_combine['CRFieldName'].replace("nutrition_assessment", "nutritional_assessment", inplace=True)

In [18]:
# merge two dataframe to one
merged_file = pd.merge(original_label_df_combine, sme_label_df_combine, how = "outer", on = ["CRFileName", "CRFieldName"])
merged_file

CRFileName Original Label             CRFieldName  \
0   1003700000000572521_11228490.pdf    16-17,37-38  nutritional_assessment   
1   1003700000000572084_11229402.pdf              0  nutritional_assessment   
2   1003700000000575266_11230267.pdf       15-16,20  nutritional_assessment   
3   1003700000000549307_11234991.pdf       55,64-67  nutritional_assessment   
4   1003700000000551125_11236376.pdf              0  nutritional_assessment   
..                               ...            ...                     ...   
66  1003700000000576763_11228491.pdf            NaN         case_management   
67  1003700000000550694_11232220.pdf            NaN         case_management   
68  1003700000000555611_11151792.pdf            NaN         case_management   
69  1003700000000570016_11091419.pdf            NaN         case_management   
70  1003700000000572521_11228490.pdf            NaN         case_management   

                  SME Label  
0   16,17,18,19,37,38,82,83  
1                       NaN  
2                     15-16  
3         55,65,67,70,71,72  
4                       NaN  
..                      ...  
66                 28,29,30  
67                    29-36  
68                    43-46  
69             101102103258  
70                   123124  

[71 rows x 4 columns]

In [19]:
# Load AWS predicted result
aws_pred_file = "Lexicon_Training_Data_New.xlsx"
aws_file = pd.read_excel(aws_pred_file)
aws_file = aws_file[['CRFileName', 'AWS Identified Pages', 'CRFieldName']]
aws_file = aws_file.dropna(subset = ['AWS Identified Pages'])

In [20]:
# Only keep the class that we want to validate from AWS predicted result, in this example we want to validate NA/DD/CM
# Since NA is not included in this predicted file, we only keep the DD and CM in this example
aws_df_combine = pd.DataFrame()
aws_df_common_dd = aws_file[aws_file['CRFieldName'] == "discharge_disposition"]
aws_df_common_ca = aws_file[aws_file['CRFieldName'] == "case_management"]
aws_df_combine = pd.concat([aws_df_combine, aws_df_common_dd, aws_df_common_ca], axis=0, ignore_index=True)

In [21]:
# Change filename to same format 
for i in range(len(aws_df_combine)):
    orig_filename = aws_df_combine['CRFileName'].iloc[i]
    filename = aws_df_combine['CRFileName'].iloc[i].split('_')[-2] + '_' + aws_df_combine['CRFileName'].iloc[i].split('_')[-1]
    aws_df_combine['CRFileName'].iloc[i] = filename

In [22]:
# Select the predicted outputs only from common files
aws_df_combine_common = aws_df_combine[aws_df_combine.CRFileName.isin(common_files)]

In [23]:
# merge all dataframes together
merged_file = pd.merge(merged_file, aws_df_combine_common, how = "outer", on = ["CRFileName", "CRFieldName"])
merged_file

CRFileName Original Label             CRFieldName  \
0   1003700000000572521_11228490.pdf    16-17,37-38  nutritional_assessment   
1   1003700000000572084_11229402.pdf              0  nutritional_assessment   
2   1003700000000575266_11230267.pdf       15-16,20  nutritional_assessment   
3   1003700000000549307_11234991.pdf       55,64-67  nutritional_assessment   
4   1003700000000551125_11236376.pdf              0  nutritional_assessment   
..                               ...            ...                     ...   
66  1003700000000576763_11228491.pdf            NaN         case_management   
67  1003700000000550694_11232220.pdf            NaN         case_management   
68  1003700000000555611_11151792.pdf            NaN         case_management   
69  1003700000000570016_11091419.pdf            NaN         case_management   
70  1003700000000572521_11228490.pdf            NaN         case_management   

                  SME Label             AWS Identified Pages  
0   16,17,18,19,37,38,82,83                              NaN  
1                       NaN                              NaN  
2                     15-16                              NaN  
3         55,65,67,70,71,72                              NaN  
4                       NaN                              NaN  
..                      ...                              ...  
66                 28,29,30                         28,29,30  
67                    29-36                              NaN  
68                    43-46                            68,69  
69             101102103258                  101,102,103,258  
70                   123124  102,105,109,113,115,124,126,127  

[71 rows x 5 columns]

In [24]:
# Save the merged file locally for further validation
merged_file.to_csv("label_revisit.csv")